# About this jupyter notebook

## Jupyter Notebook
- this is a jupyter notebook
- it consists of text cells (like the cell you're reading right now) and code cells
  - the code cells are written in the python programming language
- you can execute a cell by
  - clicking into it and pressing Shift + Enter or
  - klicking the "play" symbol at the top

## Making changes
- Unfortunately, any changes you make to this notebook will NOT be saved when closing it
- The easiest solution is to copy the code you write to a .txt-file (Word might destroy the formatting) and paste it when you continue working

## The other files
- Clicking the folder icon on the left will open (surprise) the folder that this notebook lives in
- the other file *doublet.py* contains the equations used in this notebook.
  - you can do this entire exercise without looking into *doublet.py*
  - but feel free to open it and explore
  - Since nothing is saved, even if you break something, you can simply open the link in a fresh window

## Trouble shooting
- if you changed something
- and for some reason, the result is the same as before. Or not what you think it should be:
- Sometimes the Notebook remembers something it should not remember -> Try right clicking into a cell and choose *Restart Kernel*

    


In [3]:
# Import standard packages to handle maths
import numpy as np
# Import the class "doublet" which knows how to calculate everything we need
from doublet import geoth_doub


print('Imports completed')

Imports completed


# "Creating" the doublet
In the next block of code, we will "create" the geothermal doublet. 
First, we define all the parameters that describe the doublet itself and the geolocigal and technical conditions in which it operates. 
Then we call the function *geoth_doub* (which we imported in cell 1). 

To be exact, geoth_doub() can take more parameters than the ones we define below. Check out the file doublet.py to find the whole list. But as long as we don't tell geoth_doub() which values to use, it will stick to standard values. So for simplicity, that's what we'll do.

In [4]:
# reservoir depth. The top of the reservoir in m below the surface.
res_depth = 2300

# reservoir permeability in mD
res_perm = 300

# well spacing. The distance between injector and producer in m
ws = 830

# injection temperature. the temperature (in deg Celsius) of the cold water that is being re-injected.
injT = 32

# And create the doublet object! (In the parenthesis we tell the functions which values to use.
doub1 = geoth_doub(r_d=res_depth, k_mD=res_perm, w_space=ws, T_inj=injT)


print('Doublet created')

Doublet created


# Calculations
Now we will look at the physics of the doublet. We will look at what happens in three steps*:

1. Pressure difference between inlet and outlet.
2. Velocity of the cold front.
3. Time of thermal breakthrough. (This is the time after which cold water, which has been re-injected into the reservoir, reaches the producer well. This may also be referred to as lifetime, even though technically, a tiny drop in temperature does not have to mean that a doublet can no longer be used.)

\* Disclaimer: We don't have to do these steps in this order. We could do step 3 directly, and let doublet.py handle everything in between. But by running and printing these 3 steps, we can follow the calculation and make sure it all seems reasonable.

In [10]:
# Pressure difference
dp = doub1.dp_wells()
print('Pressure difference between producer and injector:\n' + str(round(dp, 2)) + ' MPa')

# Cold front velocity
v_cold = doub1.v_coldfront()
print('\nVelocity of the coldfront:\n' + str(round(v_cold, 8)) + ' m/s')

# breakthrough
t_cold = doub1.t_breakthrough()
print('\nTime of thermal breakthrough:\n' + str(round(t_cold, 2)) + ' years')

Pressure difference between producer and injector:
1.51 MPa

Velocity of the coldfront:
1.55e-06 m/s

Time of thermal breakthrough:
16.99 years


# More Calculations
With the physics done, we can calculate how much power the doublet produces. 

1. Let p_in be the power that is required to operate the doublet. The main power-eaters are the pumps that move the water to the surface and back down.
2. Let p_out be the power produced by the doublet. The (simplified) idea here is that the power (power = energy/time) is constant over the entire lifetime of the doublet. This could mean that, if lifetime = 42 years, our doublet will continuosly produce X MWh for 42 years. After that, its output will reduce or it will simply be switched off.
3. Let p_net be p_out - p_in. This is the net "additional" power provided by the doublet. Hopefully, this number will be big!

In [12]:
# how much power (in MWh) is needed?
p_in = doub1.p_pumps()
print('\nPower consumed by the doublet:\n' + str(round(p_in, 2)) + ' MW')

# and how muchpower (in MWh) do we get out?
p_out = doub1.p_doublet()
print('\nPower produced by the doublet:\n' + str(round(p_out, 2)) + ' MW')

# net
p_net = p_out - p_in
print('\nNet power produced:\n' + str(round(p_net, 2)) + ' MW')
    


Power consumed by the doublet:
0.14 MW

Power produced by the doublet:
9.7 MW

Net power produced:
9.56 MW


# Some inspiration

In [18]:
# reservoir depth. The top of the reservoir in m below the surface.
res_depth = 2300

# reservoir permeability in mD
res_perm = np.arange(200, 650, 50)
print('List of permeabilities: ' + str(res_perm))

# well spacing. The distance between injector and producer in m
ws = np.arange(300, 1200, 100)
print('List of well spacings: ' + str(ws))

# injection temperature. the temperature (in deg Celsius) of the cold water that is being re-injected.
injT = 32

# calculate breakthrough time for every combination
breakthrough = np.empty((len(res_perm), len(ws)))

for count1, this_res_perm in enumerate(res_perm):
    for count2, that_ws in enumerate(ws):
        one_doublet = geoth_doub(r_d=res_depth, k_mD=this_res_perm, w_space=that_ws, T_inj=injT)
        one_breakthrough = one_doublet.t_breakthrough()
        breakthrough[count1, count2] = one_breakthrough
        
print('Matrix of breakthrough times:\n' +
     str(np.round(breakthrough, 2)))

List of permeabilities: [200 250 300 350 400 450 500 550 600]
List of well spacings: [ 300  400  500  600  700  800  900 1000 1100]
Matrix of breakthrough times:
[[ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]
 [ 6.14  8.19 10.23 12.28 14.33 16.37 18.42 20.47 22.52]]
